In [ ]:
!pip install pydub
!pip install mutagen

In [ ]:
!pip install sox

In [ ]:
!apt-get install sox

In [42]:
import numpy as np
from tqdm import tqdm
from mutagen.mp3 import MP3
from mutagen.easyid3 import EasyID3
from pydub import AudioSegment
from argparse import ArgumentParser
import librosa
import sox

In [ ]:
# Input methods
#waveform and sampleing rate
wav_mono, sampling_rate = librosa.load(file_name, duration=270)

#wavform and sampling rate, need wav stereo
wav_stereo, sampling_rate = librosa.load(file_name, mono=False, duration=270)

#tempo and beatframes
tempo, beat_frames = librosa.beat.beat_track(y=wav_stereo[0], sr=sampling_rate)


In [37]:
## Filtering functions useful for elevation
def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

def butter_highpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='high', analog=False)
    return b, a

def butter_highpass_filter(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = lfilter(b, a, data)
    return y

In [ ]:
# Used to provide elevations to the music to mock immersivness.
def elevation(wav_mono, tempo, sampling_rate):
    length = len(wav_mono)
    end_of_beat = int((tempo / 120) * sampling_rate)*2

    order = 6
    fs = 30.0
    i = 1
    y = np.empty(0)

    low = True

    while i < length:
        #low pass filter with cutoff decreasing
        cutoff = 10
        y = np.append(y, butter_lowpass_filter(wav_mono[i:i+end_of_beat], cutoff, fs, order))
        cutoff = 9.25
        y = np.append(y, butter_lowpass_filter(wav_mono[i+end_of_beat-1:i+2*end_of_beat], cutoff, fs, order))
        cutoff = 8.75
        y = np.append(y, butter_lowpass_filter(wav_mono[i+2*end_of_beat-1:i+3*end_of_beat], cutoff, fs, order))
        cutoff = 8
        y = np.append(y, butter_lowpass_filter(wav_mono[i+3*end_of_beat-1:i+4*end_of_beat], cutoff, fs, order))

        i += 4*end_of_beat

        #high pass filter with cutoff increasing
        cutoff = 8
        y = np.append(y, butter_highpass_filter(wav_mono[i-1:i+end_of_beat], cutoff, fs, order))
        cutoff = 8.75
        y = np.append(y, butter_highpass_filter(wav_mono[i+end_of_beat-1:i+2*end_of_beat], cutoff, fs, order))
        cutoff = 9.25
        y = np.append(y, butter_highpass_filter(wav_mono[i+2*end_of_beat-1:i+3*end_of_beat], cutoff, fs, order))
        cutoff = 10
        y = np.append(y, butter_highpass_filter(wav_mono[i+3*end_of_beat-1:i+4*end_of_beat], cutoff, fs, order))

        i += 4*end_of_beat

    return y

In [ ]:
# This function performs the left-right panning and shifting.
def convert(inputfile, outputfile='testing.wav', period=200):
    if period < 0:
        period = period*(-1)
    elif period == 0:
        period = 100
    audio = AudioSegment.from_file(inputfile, format='mp3')
    audio = audio + AudioSegment.silent(duration=150)

    eightD = AudioSegment.empty()
    pan = 0.9*np.sin(np.linspace(0, 2*np.pi, period))
    chunks = list(enumerate(audio[::100]))

    for i, chunk in tqdm(chunks, desc='Converting', unit='chunks', total=len(chunks)):
        if len(chunk) < 100:
            continue
        newChunk = chunk.pan(pan[i % period])
        eightD = eightD + newChunk

    eightD.export(outputfile, format="wav")



In [ ]:
# Sox tools used to add effects to the music file
tfm = sox.Transformer()
tfm.reverb(reverberance=25)
tfm.bass(gain_db=5, slope=0.3)
tfm.treble(gain_db=5, slope=.3)
tfm.build('/content/testing.wav','reverbtesting.wav')

In [ ]:
## Python console function lets me play the song here (song saved in the local colab memory)
from IPython.display import Audio

Audio('reverbtesting.wav')